# Find new Nets by looking at highest probability candidate from the model

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # specify which GPU(s) to be used

import torch.backends.cudnn as cudnn
cudnn.benchmark = True
from utils import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

data_path = "/media/slowSSD/Carribean_Challenge/data/fish"
path_to_visuals = "/media/austausch/popien/MachineLearning/CharlesProject/FishProject/visuals"

## Predict new random test images and find new nets
- Sort new random images by descending probability of fish net
- Visualize, find new nets and copy their coordinates into "fish_coordinates" in crawling.py 

In [20]:
# sz = (450, 432)
sz = (900, 864)
random_crop = 448
test_images = glob.glob("{}/no_nets/*.png".format(data_path = "/media/slowSSD/Carribean_Challenge/data/fish"))
print("{:3} test images".format(len(test_images)))

7515 test images


In [21]:
epochs = 1000
print_every_x_batches = 1000
lr = 0.001
arch = "resnet18"

val_transforms = []
val_transforms.append(albumentations.Resize(sz[1], sz[0], interpolation=cv2.INTER_CUBIC))
val_dataset = SalmonDataset(images=test_images, test=True, transforms=albumentations.Compose(val_transforms)) 

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=10, pin_memory=True)

In [22]:
# model_name = "resnet18_88nets_6405wo_val_v1_resize_900x864_rc_448_pat3"
model_name = "resnet18_94nets_6401wo_val_v1_resize_900x864_rc_448_pat3"

In [18]:
preds = predict(model_name, val_loader, infrared=False, verbose=True)

resnet18_94nets_6401wo_val_v1_resize_900x864_rc_448_pat3 with loss 0.0266 at epoch 27


100%|██████████| 118/118 [01:48<00:00,  1.09it/s]


In [ ]:
how_many = 30
count = 0
skip = 0

# for j in np.argsort(preds)[::-1][20:]:
for j in np.argsort(preds)[::-1]:
    count += 1
    if count<skip: continue
    rand_int = j
    f, axarr = plt.subplots(1, 2, figsize=(30, 20))
#     plt.figure(figsize=(12, 12))
    img = Image.open(test_images[rand_int])
    axarr[0].imshow(img)
    axarr[0].set_title(test_images[rand_int])
    sample = val_dataset[rand_int]
    axarr[1].imshow(np.transpose((sample["image"].numpy()*255).astype(np.uint8), (1,2,0)))
#     axarr[1].set_title("{:.1f}% for Net, ({:.1f}% old model)".format(preds2[rand_int]*100, preds[rand_int]*100))
    axarr[1].set_title("{:.1f}% for Net".format(preds[rand_int]*100))
    print(test_images[rand_int].split("/")[-1])
    plt.show()
    if count>how_many+skip:
        break

In [1]:
# for path in delete_from_no_nets:
#     print(path, end="...")
#     try:
#         os.remove("/media/slowSSD/Carribean_Challenge/data/fish/no_nets/{}".format(path))
#         print("yes")
#     except:
#         print("no")

## Visualize

### In QGIS

In [3]:
coords = [j.split("/")[-1].split("_")[1] for j in glob.glob("{}/no_nets/*".format(data_path))]
no_net_coords = []
for coord in coords:
    lat, long = map(float, coord.split(","))
    no_net_coords.append((lat, long))
len(no_net_coords)

7514

In [4]:
no_net_coords = pd.DataFrame({"lat": [k for k,v in no_net_coords], "long": [v for k,v in no_net_coords]})

net_coords = pd.DataFrame({"lat": [k for k,v in fish_coordinates], "long": [v for k,v in fish_coordinates]})

In [6]:
os.makedirs(path_to_visuals, exist_ok=True)
gdf = gpd.GeoDataFrame({"class": ["net_present"]*len(net_coords) + ["no_net"]*len(no_net_coords), "geometry": gpd.points_from_xy(net_coords.long, net_coords.lat) + gpd.points_from_xy(no_net_coords.long, no_net_coords.lat)})
gdf.crs = {"init": "epsg:{}".format(4326)}
gdf.to_file("{}/output.shp".format(path_to_visuals))

### Print Net coordinates to copy paste into http://www.copypastemap.com/index.html

In [3]:
coords = pd.DataFrame({"lat": [k for k,v in fish_coordinates], "long": [v for k,v in fish_coordinates]})
for j in range(len(coords)):
    print("{}\t{}".format(coords["lat"].iloc[j], coords["long"].iloc[j]))

52.591901	-128.521975
50.882396	-127.49693
50.878538	-126.902387
50.854201	-126.758715
50.850617	-126.716857
50.847869	-126.319231
50.837714	-126.663131
50.832302	-127.520585
50.831666	-126.597147
50.796923	-126.495746
50.749069	-127.683667
50.746123	-127.647787
50.743125	-127.448952
50.733599	-127.444264
50.707619	-126.664039
50.69787	-126.25668
50.670949	-126.476713
50.656792	-126.665785
50.653974	-126.28905
50.649603	-126.618208
50.618472	-126.704306
50.607944	-126.363262
50.601005	-126.348642
50.566306	-126.274258
50.594348	-127.571985
50.488321	-125.35654
50.4857	-127.876225
50.474602	-127.787872
50.473781	-125.808549
50.468923	-127.835091
50.458708	-127.890031
50.425624	-125.305282
50.414876	-125.768263
50.41463	-125.659615
50.409818	-125.700438
50.408549	-125.340233
50.392891	-125.362241
50.349968	-125.34287
50.324312	-125.26048
50.309412	-125.316563
50.302343	-125.338198
50.286035	-125.349444
50.178898	-125.327057
50.177378	-125.344332
50.046627	-127.296477
50.037817	-127.17600